In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
from models import *
from function import *
from tqdm import tqdm

!nvidia-smi
print("CUDA Available : ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Wed Sep 20 19:32:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.34                 Driver Version: 537.34       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:2B:00.0  On |                  N/A |
|  0%   54C    P8              43W / 350W |  12362MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Argument
batch_size = 128
learning_rate = 1e-6
num_epochs = 50
momentum = 0.9
weight_decay = 1e-9
class_num = 10
#class_num = 2 # clean or adversarial
output_path = 'test'

if not os.path.isdir(output_path):
    os.mkdir(output_path)

In [3]:
# Preprocessing
test_tfm = transforms.Compose([
    transforms.ToTensor(),
])
toPIL = transforms.ToPILImage()

In [4]:
# Cifar10
train_set = torchvision.datasets.CIFAR10(root='../../dataset/cifar10_dataset', train=True, download=True, transform=test_tfm)
test_set = torchvision.datasets.CIFAR10(root='../../dataset/cifar10_dataset', train=False, download=True, transform=test_tfm)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = Unet(n_channels = 3, n_classes = 3, bilinear = False)
model = model.to(device)
write_path = 'result'

target_model = ResNet18()
target_model = target_model.to(device)
adversary = LinfPGDAttack(target_model)



if not os.path.isdir(write_path):
    os.mkdir(write_path)
    
optimizer = optim.RMSprop(model.parameters(), lr = learning_rate, weight_decay = weight_decay, momentum = momentum, foreach = True)
criterion = diceLoss(3)
#criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    #idx = 0
    # clean train
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        adv_imgs = adversary.perturb(imgs, labels)
        
        output = model(imgs)
        loss = criterion(output, imgs)
        
        output = model(adv_imgs)
        loss += criterion(output, imgs)
        
        
        epoch_loss += loss
        
        loss.backward()
        optimizer.step()

    print(f'epoch : {epoch}  Training | loss = {epoch_loss / imgs.shape[0]}')
    model.eval()
    eval_loss = 0
    idx = 0
    
    for batch in tqdm(test_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        adv_imgs = adversary.perturb(imgs, labels)
        with torch.no_grad():
            output = model(adv_imgs)
            loss = criterion(output, imgs)
            eval_loss += loss
        
        for i in range(imgs.size(0)):
            img = toPIL(output[i].squeeze(0)).convert('RGB')
            img.save(f'{write_path}/img_{idx}.png')
            idx += 1
            
    print(f'epoch : {epoch}  Evaluating | loss = {eval_loss / imgs.shape[0]}')
torch.save(model.state_dict(), './pretrain/denoiser_cifar10.pth')

100%|██████████| 391/391 [01:30<00:00,  4.31it/s]


epoch : 0  Training | loss = 2.8574464321136475


100%|██████████| 79/79 [00:28<00:00,  2.77it/s]


epoch : 0  Evaluating | loss = 0.5939290523529053


100%|██████████| 391/391 [01:23<00:00,  4.70it/s]


...

epoch : 48  Evaluating | loss = 0.04306329786777496


100%|██████████| 391/391 [01:22<00:00,  4.73it/s]


epoch : 49  Training | loss = 0.06171285733580589


100%|██████████| 79/79 [00:21<00:00,  3.66it/s]


epoch : 49  Evaluating | loss = 0.03361659124493599


In [6]:
target_model = ResNet18()
target_model = nn.DataParallel(target_model).to(device)
checkpoint = torch.load('pretrain/basic_training')
target_model.load_state_dict(checkpoint['net'])
target_model.eval()
adversary = LinfPGDAttack(target_model)

In [7]:
accuracy = 0
total = 0
idx = 0

new_path = 'cifar_test'
if not os.path.isdir(new_path):
    os.mkdir(new_path)

for batch in tqdm(test_loader):
    imgs, labels = batch
    imgs, labels = imgs.to(device), labels.to(device)
    adv_imgs = adversary.perturb(imgs, labels)
    with torch.no_grad():
        output = model(adv_imgs)
    
    logits = target_model(output)
    _, predicted = logits.max(1)
    accuracy += predicted.eq(labels).sum().item()
    total += imgs.size(0)
    
    for i in range(imgs.size(0)):
        img = toPIL(output[i].squeeze(0)).convert('RGB')
        img.save(f'{new_path}/img_{idx}.png')
        idx += 1
            
print(f'epoch : {epoch}  Evaluating | Accuracy = {accuracy / total}')

100%|██████████| 79/79 [00:21<00:00,  3.68it/s]

epoch : 49  Evaluating | Accuracy = 0.6814
